In [1]:
import time
print("Dette programmet tar XML-filer med Ibsenstruktur og gjør dem om til rentekstfiler. Trykk STOPP nå om du ikke vil kjøre.")
time.sleep(5)

Dette programmet tar XML-filer med Ibsenstruktur og gjør dem om til rentekstfiler. Trykk STOPP nå om du ikke vil kjøre.


KeyboardInterrupt: 

In [5]:
from bs4 import BeautifulSoup # Hent BeautifulSoup-modulen (https://www.crummy.com/software/BeautifulSoup/)
import re # Requests. Brukes i noen regex-funksjoner, gitt
import os # Filsystem for lasting og lagring
import codecs # Dekoder for ASCII
import csv # CSV-leser for ASCII

In [6]:
#Etabler liste over koder og ascii

file = open('translation.csv') # Egendefinert fil fra DTD. &nbsp, &shy og &tu20 er manuelt oversatt til [mellomrom], [-] og [mellomrom].
# Andre diverse entiteter som manglet er også oversatt.
csvreader = csv.reader(file) # Importer  filen over
asciis = [] # Array for unicode-greier (eks. \u0023)
codes = [] # Array for ting som &nbsp - "html"-koder
for row in csvreader: # For hver rad
    htmlcode = row[0] # Html-tagger (de som er synlige i teksten)
    unicode = row[1] # Unicode (eg \u0023)
    ASCII = codecs.decode(unicode, 'unicode_escape') # Gjør om til ASCII
    asciis.append(ASCII) # Legg till i array
    codes.append(htmlcode)

#Hvilke tagger skal utraderes MED innhold?
tags_to_erase = ["handnote","teiheader","front","lb","figure","pb","note","anchor"]

# GLOB GLOB GOBGALAB
import glob
paths = []
for path in glob.glob('xml_src/**/*.xml', recursive=True):
    paths.append(path)
paths # Liste over stier til filer i SRC
errors = []
outputs = []
for path in paths:
    path = path.replace('xml_src','xml_output')
    outputs.append(path)
outputs # Liste over stier til nye filer

['xml_output\\brev\\B1844-1871ht.xml',
 'xml_output\\brev\\B1871-1879ht.xml',
 'xml_output\\brev\\B1880-1889ht.xml',
 'xml_output\\brev\\B1890-1905ht.xml',
 'xml_output\\dikt\\BlaDigt8191.xml',
 'xml_output\\dikt\\Di01.xml',
 'xml_output\\dikt\\Di41110a.xml',
 'xml_output\\dikt\\Di41113h.xml',
 'xml_output\\dikt\\Di42622.xml',
 'xml_output\\dikt\\Di71.xml',
 'xml_output\\dikt\\Di75.xml',
 'xml_output\\dikt\\Di79.xml',
 'xml_output\\dikt\\Di81928.xml',
 'xml_output\\dikt\\Di82.xml',
 'xml_output\\dikt\\Di86.xml',
 'xml_output\\dikt\\Di91.xml',
 'xml_output\\dikt\\Di96.xml',
 'xml_output\\dikt\\DiFU.xml',
 'xml_output\\dikt\\Diktht.xml',
 'xml_output\\drama\\BE\\BE428691_KBK.xml',
 'xml_output\\drama\\BE\\BE428691_KBK_gml.xml',
 'xml_output\\drama\\BE\\BE428691_KBK_original.xml',
 'xml_output\\drama\\BE\\BEht.xml',
 'xml_output\\drama\\BE\\BEht_noter.xml',
 'xml_output\\drama\\BE\\BEht_original.xml',
 'xml_output\\drama\\BE\\BEht_ut.xml',
 'xml_output\\drama\\Br\\Br03.xml',
 'xml_output\

In [9]:
thing = paths[3]
with open(thing, "r", encoding="utf-8") as file: # Åpne stien, les, med encoding UTF8
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content) # Variabel content får innholdet
content = BeautifulSoup(content, from_encoding="UTF-8") # Content er content som bs4-objekt...
content

C:\Users\lokes\Anaconda3\lib\site-packages\bs4\__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


<?xml version="1.0" encoding="UTF-8" standalone="yes"?><!DOCTYPE TEI>
<html><body><p>
%HIS_entiteter;
]&gt;
<?xml-model href="http://www.edd.uio.no/ibsen/schema/tei_his.rnc" type="application/relax-ng-compact-syntax"?>
<tei xmlns="http://www.tei-c.org/ns/1.0" xmlns:his="http://www.example.org/ns/HIS">
<teiheader xml:lang="nob">
<filedesc>
<titlestmt>
<title level="a" type="sub">Brev fra perioden 1890-1905</title>
<title level="s" type="main">Henrik Ibsens skrifter 2010</title>
<title level="s" type="copyright">© Henrik Ibsens skrifter</title>
<author>Henrik Ibsen</author>
<funder>&amp;funder;</funder>
<editor>&amp;editor;</editor>
<respstmt type="editor"><resp>Hovedredaktør</resp><name>Narve Fulsås</name></respstmt>
<respstmt type="editing"><resp>Edering</resp><name>Christian Janss (ansv.), Aina Nøding, Ellen Nessheim Wiger</name>
</respstmt>
<respstmt type="encoding"><resp>Kollasjonering og koding</resp><name>Bodil Aurstad, Gry Berg, Aasta Marie Bjorvand Bjørkøy, Oliver Blomqvist, Nin

In [ ]:
# ACTION
i = 0
x = 0
y = 0
for thing in paths: # For hver sti (med endestasjon fil) i listen over stier
    # Filhåndtering I - Les og last xml-filer inn i minne
    with open(thing, "r", encoding="utf-8") as file: # Åpne stien, les, med encoding UTF8
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # Combine the lines in the list into a string
        content = "".join(content) # Variabel content får innholdet
    
    # Filhåndtering II - Lag ny sti
    frags = outputs[i].split('\\') # Splitt outputstrengen for sti
    if "xml" in frags[2]: # Hvis filen er 2 ned, altså vi finner xml-filen her (eks. xml_src\dikt\~.xml)
        dirName = frags[0]+"\\"+frags[1] # Dirname blir 2 dyp (eks. xml_output\dikt\)
    else: # OBS!! OBS!! Hvis den IKKE er 2 ned ANTAR vi at den er 3 ned - som i DRAMA!!!
        dirName = frags[0]+"\\"+frags[1]+"\\"+frags[2] # Dirname blir 3 ned (xml_output\drama\XYZ)
        archDir = frags[0]+"\\"+frags[1] # FAILSAFE for DRAMA: Drama er ett hakk lengre ned i hierarkiet. Skap mappe for å ta drama
        #OBS - koden her ANTAR at senteret ALLTID følger samme praksis. Koden må endres om denne endres.
        if not os.path.exists(archDir): # Hvis subfolder ikke finnes (eks.\drama\BE\)
            os.mkdir(archDir) # Skap den
            #print("Created",archDir,"for deep path")
    outputs[i] = outputs[i][:-4] # Strip 4 tegn fra slutten av hver streng for å få bort .xml (4. 3X 2M 1L)
    # Create target Directory if don't exist
    if not os.path.exists(dirName):
        os.mkdir(dirName)
        #print("Created",dirName,"from full path")
    name = outputs[i]+"_SANERT.txt" # Outputfil får navnet X_SANERT.txt
    
    # XML til TXT - Det er her vi skal sette inn X FOR EACH for å redigere XML-filene.
    
    content = BeautifulSoup(content, from_encoding="UTF-8") # Content er content som bs4-objekt...
    
    # Hent strukturer vi skal beholde
    try:
        titlestatement = content.find("titlestmt") # Finn title statement
        titlestatement.extract() # Hent (og fjern) title statement-delen
    except:
        print("Ingen title statement.")

    try:
        filedescription = content.find("filedesc") # Finn filbeskrivelsen
        filedescription.extract() # Hent (og fjern) filbeskrivelsen
    except:
        print("Ingen filbeskrivelse.")
    # Slett tagger vi ikke skal beholde
    try:
        for tag in tags_to_erase:
            to_remove = content.find_all(tag) # Finn alle instanser av taggen
            for remove in to_remove: # For hver av disse
                remove.decompose() # Fjern taggen og innholdet helt
    except:
        print("Oi, det var det en feil med",tag,remove)

    # Taggen his:hisdel rend="overstrike" - skal denne slettes? nei! Legg inn blanktegn eller annen formatering som markerer.
    
    content = str(content) # CONTENT ER NÅ STRING - STRINGMETODER FUNGERER, BS4-METODER FUNGERER IKKE
    
    z = 0 # Tell fra 0 i z
    for code in codes: # For hver kode
        if code in content: # Hvis koden finnes i teksten
            content = content.replace(code,asciis[z]) # Erstatt koden med ascii
            #print('Erstattet',code,'med',asciis[z]) # Print bekreftelse
        z+=1 # Neste kode/ascii etter å ha sjekket hver kode
    
    content = content.replace("\n"," ") # Slett newline og erstatt med mellomrom
    # Ordinær RegEx-vask
    content = content.replace(".</l>",".") # Fjern slutten på L og erstatt med et punktum der det er punktum-.. vent.
    # Hvorfor gjorde jeg det over, igjen...?
    content = re.sub('<[^<]+>', "", content) # Slett alle resterende tagger UTEN å røre innholdet
    content = re.sub("'“'", "", content) # Korriger enkelte instanser av '“'
    content = content.replace("  "," ") # Prøv å fjerne dobbelt mellomrom
    
    # Filhåndtering III - Skriv til disk
    try: # Prøv å skrive til disk
        f = open(name, "w", encoding="utf-8") # Ikke glem encoding="utf-8", sånn som jeg gjorde.
        f.write(content)
        f.close()
        print("Opprettet",name,"i",dirName)
        x+=1
    except: # Hvis vi møter på feil, rapporter kilden
        print("FILFEIL!",thing)
        f = open(name+"_ERROR", "w", encoding="utf-8")
        f.write("å neeeiii")
        f.close()
        errors.append(name)
        y+=1
    i+=1
#print(content)
print("\n",x,"files parsed to text.",y,"files failing to parse.")
if y > 0:
    face = ":("
else:
    face = ":)"

In [ ]:
print("Gjorde om",i+1,"dokumenter til rentekst fra",len(paths)+1,"XML-filer. Vi fikk",y,"encoding-feil.",face)
if errors:
    errors

~~UnicodeEncodeError: 'charmap' codec can't encode character '\u203b' in position 1180: character maps to undefined. Denne karakteren her: ※ (Fixed)~~

~~Ok, fra 3 feil til 1 etter å ha konvertert "&ergo;" til "ɔ"!~~
Header
FIX: Glemte å manuelt sette output til encoding="utf-8"... men nå skriver alle filer!

TODO: Mange korreksjoner og sånne ting som gjemmer seg blant teksten i tags.

TODO: De variablene under må dekonstrueres og deler av dem må stappes inn igjen.

In [2]:
filedescription

NameError: name 'filedescription' is not defined

In [3]:
titlestatement

NameError: name 'titlestatement' is not defined

In [4]:
content

NameError: name 'content' is not defined